In [ ]:
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip install mlxtend
!pip install refractio 

In [1]:
import os
from snowflake.snowpark.session import Session
user = os.getenv("user")
warehouse = os.getenv("Warehouse")
schema= os.getenv("SCHEMA")
database = os.getenv("Database")
role =  os.getenv("ROLE")
account =  os.getenv("Account")
password= os.getenv("Password")

connection_params = dict(user=user, 
                         password=password, 
                         account=account, 
                         warehouse=warehouse, 
                         database=database,
                         schema=schema, 
                         role=role)

session = Session.builder.configs(connection_params).create()

session.sql('use warehouse {};'.format(warehouse)).collect()

session.sql('use database {};'.format(database)).collect()

session.sql('use schema {}.{};'.format(database, schema)).collect()

[Row(status='Statement executed successfully.')]

In [2]:
dataset=session.sql("select * from RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.product_sales").to_pandas()

In [3]:
from refractio import snowflake
#snowflake.get_connection("RETAIL_MKT_BSKT_CXN")
#dataset = snowflake.execute_query(f"RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.product_sales")
dataset

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


,CUSTOMER_ID,CUSTOMER_NAME,SEGMENT,CITY,STATE,COUNTRY,MARKET,REGION,ORDER_DATE,ORDER_ID,PRODUCT_ID,PRODUCT,QUANTITY,UNIT_PRICE,SHIP_MODE,TOTAL_SALES
0,MH-7455,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,2022-04-30,BU-2013-7380,10002439,shrimp,167,74.69,Standard Class,12473.23
1,MH-7455,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,2022-04-30,BU-2013-7380,10000353,almonds,278,15.28,Standard Class,4247.84
2,MH-7455,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,2022-04-30,BU-2013-7380,10000994,avocado,324,46.33,Standard Class,15010.92
3,MH-7455,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,2022-04-30,BU-2013-7380,10000475,vegetables mix,124,58.22,Standard Class,7219.28
4,MH-7455,Mark Hamilton,Consumer,Burgas,Burgas,Bulgaria,EMEA,Middle Eastern,2022-04-30,BU-2013-7380,10004632,cottage cheese,148,73.56,Standard Class,10886.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71656,CG-12040,Catherine Glotzbach,Home Office,Camag?ey,Camag?ey,Cuba,LATAM,Caribbean,2024-06-29,MX-2013-165708,10002981,onions,10,19.15,Standard Class,191.50
71657,EG-13900,Emily Grady,Consumer,Puteaux,Ile-de-France,France,EU,Central,2024-06-29,US-2013-150861,10002981,onions,10,19.15,First Class,191.50
71658,Dp-13240,Dean percer,Home Office,Westfield,New Jersey,United States,US,East,2024-06-30,MX-2013-118577,10002981,onions,10,19.15,Standard Class,191.50
71659,RB-9330,Randy Bradley,Consumer,Donets'k,Donetsk,Ukraine,EMEA,Middle Eastern,2024-06-30,UP-2013-6200,10002981,onions,10,19.15,Standard Class,191.50


In [4]:
import pandas as pd
product_set=dataset.groupby("ORDER_ID")["PRODUCT"].apply(set).to_dict()

In [5]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
List_order_id=list(filter_basket.keys())
List_product_id=[list(values) for values in filter_basket.values()]
dict1={"ORDER_ID":List_order_id,"PRODUCT":List_product_id}
df1=pd.DataFrame(dict1)
df1

,ORDER_ID,PRODUCT
0,AE-2013-1130,"[tomato juice, cooking oil, eggs, burgers, fro..."
1,AE-2013-1530,"[turkey, cottage cheese, cooking oil, chocolat..."
2,AE-2014-2840,"[baking powder, newspapers, sugar, fruit/veget..."
3,AE-2014-4120,"[other vegetables, red/blush wine]"
4,AG-2013-2840,"[avocado, green tea, grated cheese, french fries]"
...,...,...
12116,ZI-2014-6740,"[tropical fruit, canned beer, red/blush wine, ..."
12117,ZI-2014-6750,"[eggs, french fries, coffee]"
12118,ZI-2014-7160,"[other vegetables, meat, chicken, whole milk]"
12119,ZI-2014-9540,"[curd, soda, whipped/sour cream, canned vegeta..."


In [6]:
# Create a list of unique items
unique_items = sorted(set(item for sublist in df1['PRODUCT'] for item in sublist))
 
# Initialize an empty dictionary to hold binary values for each item
binary_data = {item: [] for item in unique_items}
 
# Iterate over each row in the original DataFrame
for bought_together in df1['PRODUCT']:
    # Initialize a binary row for the current transaction
    binary_row = {item: 0 for item in unique_items}
    # Set the value to 1 if the item was bought in the transaction
    for item in bought_together:
        binary_row[item] = 1
    # Append the binary row to the binary_data dictionary
    for item, value in binary_row.items():
        binary_data[item].append(value)
 
#Create a DataFrame from the dictionary
binary_df = pd.DataFrame(binary_data)
 
# Concatenate with the original DataFrame
binary_df2 = pd.concat([df1['ORDER_ID'], binary_df], axis=1)
 
# Display the binary DataFrame
binary_df

,UHT-milk,almonds,avocado,baking powder,beef,berries,beverages,black tea,bottled beer,bottled water,...,vegetables mix,waffles,whipped/sour cream,white bread,white wine,whole milk,whole wheat pasta,whole wheat rice,yogurt,yogurt cake
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12116,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12117,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
12119,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [7]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
# Load your dataset
frequent_itemsets = apriori(binary_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
# Filter rules based on desired metrics
#rules
filtered_rules = pd.DataFrame(rules[(rules['confidence'] > 0.01) & (rules['lift'] > 0.05)])
#
# # Print the filtered rules
filtered_rules

/tmp/pip_packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.041003,0.151225,0.016500,0.402414,2.661029,0.010300,1.420340,0.650894
1,(other vegetables),(beef),0.151225,0.041003,0.016500,0.109111,2.661029,0.010300,1.076449,0.735419
2,(beef),(rolls/buns),0.041003,0.140830,0.011550,0.281690,2.000215,0.005776,1.196099,0.521434
3,(rolls/buns),(beef),0.140830,0.041003,0.011550,0.082015,2.000215,0.005776,1.044676,0.582019
4,(beef),(root vegetables),0.041003,0.086297,0.014438,0.352113,4.080266,0.010899,1.410281,0.787195
...,...,...,...,...,...,...,...,...,...,...
829,"(yogurt, whole milk)",(tropical fruit),0.046861,0.083079,0.012953,0.276408,3.327057,0.009060,1.267180,0.733822
830,"(tropical fruit, whole milk)",(yogurt),0.035476,0.109562,0.012953,0.365116,3.332511,0.009066,1.402522,0.725670
831,(yogurt),"(tropical fruit, whole milk)",0.109562,0.035476,0.012953,0.118223,3.332511,0.009066,1.093841,0.786047
832,(tropical fruit),"(yogurt, whole milk)",0.083079,0.046861,0.012953,0.155909,3.327057,0.009060,1.129190,0.762807


In [8]:
frequent_itemsets

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.026318,(UHT-milk)
1,0.012540,(almonds)
2,0.020213,(avocado)
3,0.014025,(baking powder)
4,0.041003,(beef)
...,...,...
485,0.011138,"(root vegetables, rolls/buns, whole milk)"
486,0.013695,"(rolls/buns, yogurt, whole milk)"
487,0.010148,"(root vegetables, tropical fruit, whole milk)"
488,0.012540,"(root vegetables, yogurt, whole milk)"


In [9]:
rules

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(beef),(other vegetables),0.041003,0.151225,0.016500,0.402414,2.661029,0.010300,1.420340,0.650894
1,(other vegetables),(beef),0.151225,0.041003,0.016500,0.109111,2.661029,0.010300,1.076449,0.735419
2,(beef),(rolls/buns),0.041003,0.140830,0.011550,0.281690,2.000215,0.005776,1.196099,0.521434
3,(rolls/buns),(beef),0.140830,0.041003,0.011550,0.082015,2.000215,0.005776,1.044676,0.582019
4,(beef),(root vegetables),0.041003,0.086297,0.014438,0.352113,4.080266,0.010899,1.410281,0.787195
...,...,...,...,...,...,...,...,...,...,...
829,"(yogurt, whole milk)",(tropical fruit),0.046861,0.083079,0.012953,0.276408,3.327057,0.009060,1.267180,0.733822
830,"(tropical fruit, whole milk)",(yogurt),0.035476,0.109562,0.012953,0.365116,3.332511,0.009066,1.402522,0.725670
831,(yogurt),"(tropical fruit, whole milk)",0.109562,0.035476,0.012953,0.118223,3.332511,0.009066,1.093841,0.786047
832,(tropical fruit),"(yogurt, whole milk)",0.083079,0.046861,0.012953,0.155909,3.327057,0.009060,1.129190,0.762807


In [10]:
filtered_rules = filtered_rules.rename(columns={'antecedents': 'product' , 'consequents': 'recommendation' , 'antecedent support': 'product_support','consequent support': 'recommendation_support' })


/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
import json
filtered_rules['product'] = list(filtered_rules['product'])
file = 'filtered_rules.csv'
filtered_rules.to_csv(file , index=False)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
session.sql(f"PUT file://{file} @RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.PRODUCT_STAGE").collect()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[Row(source='filtered_rules.csv', target='filtered_rules.csv.gz', source_size=177579, target_size=0, source_compression='NONE', target_compression='GZIP', status='SKIPPED', message='')]

In [ ]:
#pip install fosforml

In [ ]:
#from fosforml import *

In [ ]:
input_file = pd.read_csv("Order_Inventory.csv")
input_file

In [ ]:
req = requests.Request()
req.file = "Order_Inventory.csv"

In [ ]:
pd.read_csv(req.file)

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

@scoring_func
def score(model, request):
    import json
    payload = request.json["payload"]
    if isinstance(request.json["payload"],str):
        payload_data = eval(payload)
        if isinstance(payload_data['Geography'], str):
            data_json = eval(payload)
            data = pd.DataFrame([data_json])
            prediction = pd.DataFrame(model.predict(data))
            return prediction[0].to_list()[0]
        elif isinstance(payload_data['Geography'], dict):
            data = pd.DataFrame(eval(payload))
            prediction = pd.DataFrame(model.predict(data))
            return prediction[0].tolist()
        elif isinstance(payload_data['Geography'], list):
            data = pd.DataFrame(payload_data)
            prediction = model.predict(data)
            return prediction.tolist()
    return "This method is not allowed"

payload = str(X.iloc[158].to_dict())
req = requests.Request()
req.json= {'payload': payload}
print({'payload': payload})




In [ ]:
snowflake.get_connection('RETAIL_MKT_BSKT_CXN')